In [1]:
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import feature_handler as handler

## Import data
- name : stock list
- date : date we collected
- clear : stock data we crawl dowm

In [2]:
Stock_list = pd.read_csv('name.csv')['name']
date = pd.read_csv('date.csv')['date']
Stock = pd.read_csv('clear.csv')
Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
Stock = Stock.reset_index(drop=True)

## Build all or Update
- update flag for solving time rebuild all
- Stock_PRE : previous data for model training

In [3]:
update = False
#update = True
if update :
    Stock_PRE = pd.read_csv('model.csv')
    finall_date = max(Stock_PRE.date)
    Stock_PRE

## building feature for model

In [4]:
list_1 = []
list_2 = []
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    handler.前高(s,list_1)
    handler.波段前高(s,list_2)
Stock['年前高'] = list_1
Stock['季前高'] = list_2

In [5]:
if update :
    start_date = date[date[date.values == finall_date ].index + 80].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [ ]:
handler.紅棒(Stock ,"實紅棒","紅棒天線")

In [ ]:
handler.黑棒(Stock , "實黑棒","黑棒天線")

In [ ]:
handler.避雷針(Stock , "上避雷針","下避雷針")

In [78]:
handler.連續(Stock , "連漲跌")

list index out of range


In [ ]:
handler.季均漲跌天數(Stock , 60)

In [ ]:
feature = 'MA60'
label = '收盤價'
test_ = []
day = int(''.join([x for x in feature if x.isdigit()]))
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    handler.增加技術指標MA(s,day,label,test_)        
Stock[features = test_

In [ ]:
if update :
    start_date = date[date[date.values == finall_date ].index + 30].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [6]:
# adding MA feature
features =['MA5', 'MA10', 'MA20', 'buy5', 'buy10','外5', '外10', '投5','投10', '自5', '三大5', '三大10' ]
labels = ['收盤價', '收盤價','收盤價', '成交股數', '成交股數','外資進出','外資進出', '投信進出', '投信進出', '自營進出', '三大', '三大'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
    for i in Stock_list :
        s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
        handler.增加技術指標MA(s,day,labels[l],test_)        
    Stock[features[l]] = test_

In [7]:
handler.day_Volatility(Stock, '日振幅')

In [8]:
handler.Implied_Volatility(Stock,'波動率',20)

In [9]:
if update :
    start_date = date[date[date.values == finall_date ].index + 10].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [10]:
handler.MA交叉型態(Stock)

-1


In [11]:
handler.KD(Stock)

In [12]:
handler.KD交叉型態(Stock)

ValueError: Length of values (447903) does not match length of index (447904)

In [ ]:
handler.KD交叉型態2(Stock)

In [15]:
handler.漲跌(Stock)

ValueError: Length of values (447903) does not match length of index (447904)

In [18]:
handler.增加技術指標RSI(Stock)

division by zero zero 0
division by zero zero 1
division by zero zero 2
division by zero zero 3
division by zero zero 4


In [25]:
feature = ['MA5 slope','MA10 slope','MA60 slope','ratio slope']
label = ['MA5','MA10','MA60','ratio' ]
for i in range(len(label)) :
    handler.power_Slope(Stock , feature[i] , label[i])

In [26]:
feature = ['margin slope','short slope','三大 slope','外資5 slope','投信5 slope','自營 slope','buy5 slope','buy10 slope','MA5 acc']
label = ['融資餘','融卷餘','三大5','外5','投5', '自5', 'buy5','buy10','MA5 slope']
for i in range(len(label)) :
    handler.Slope(Stock , feature[i] , label[i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [27]:
handler.RSI鈍化(Stock)

In [28]:
feature = ['外5金','投5金','自5金','融資金','融卷金']
label1 = ['收盤價','收盤價','收盤價','收盤價','收盤價']
label2 = ['外5','投5','自5','融資買','融卷賣']
for i in range(len(feature)):
    handler.intercept(Stock , feature[i] , label1[i] ,label2[i])

In [33]:
feature = ['D5','D60','D三大','D_upper_day']
label1 = ['收盤價','收盤價','三大5','收盤價']
label2 = ['MA5','MA60','三大10','最高價']
for i in range(len(feature)):
    handler.distance(Stock , feature[i] , label1[i] ,label2[i])

In [34]:
feature = ['UD5','UD60','UD前高壓力','UD波段壓立']
label1 = ['收盤價','收盤價','收盤價','收盤價']
label2 = ['MA5','MA60','年前高','季前高']
for i in range(len(feature)):
    handler.Updown_state(Stock , feature[i] , label1[i] ,label2[i])

In [35]:
handler.均線糾結(Stock)

C:\Users\kevin\Desktop\Stock\feature_handler.py:502: RuntimeWarning: invalid value encountered in double_scalars
  d = min(MA5,MA10,MA20) / max(MA5,MA10,MA20)
C:\Users\kevin\Desktop\Stock\feature_handler.py:503: RuntimeWarning: invalid value encountered in double_scalars
  d2 = min(MA5,MA10,MA20,MA60) / max(MA5,MA10,MA20,MA60)


In [38]:
三陽開泰 (Stock)

max() arg is an empty sequence 0


In [39]:
handler.突破前高(Stock)

In [40]:
handler.突破整理(Stock)

max() arg is an empty sequence 0
max() arg is an empty sequence 1
max() arg is an empty sequence 2
max() arg is an empty sequence 3
max() arg is an empty sequence 4
max() arg is an empty sequence 5
max() arg is an empty sequence 6
max() arg is an empty sequence 7
max() arg is an empty sequence 8
max() arg is an empty sequence 9


In [41]:
feature = ['均買量','R_散戶']
label1 = ['成交金額','三大 slope']
label2 = ['成交筆數','margin slope']
for i in range(len(feature)):
    handler.Ratio(Stock , feature[i] , label1[i] ,label2[i])

In [80]:
handler.超漲跌(Stock)

In [42]:
days = [5,3,1]
uppers = [0.1 , 0.05 ,0.03 ]
for i in range(len(days)):
    handler.shift_price(Stock, days[i] , uppers[i])

-5 0
-5 0
-4 1
-4 1
-3 2
-3 2
-2 3
-2 3
-1 4
-1 4
-3 0
-3 0
-2 1
-2 1
-1 2
-1 2
-1 0
-1 0


In [ ]:
if update :
    Stock_new = Stock[Stock.date > finall_date].reset_index(drop = True)
    Stock_new
    Stock = Stock_PRE.append(Stock_new)
    Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
    Stock = Stock.reset_index(drop=True)

In [85]:
Stock_index = pd.read_csv('Market_index.csv')
Stock_index.sort_values(by=['Index','date'] , ascending=True, axis =0 , inplace=True)
index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
Stock_index = Stock_index.reset_index(drop=True)


features =['MA5', 'MA20' ]
labels = ['收盤', '收盤'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
    for i in index_set:
        s = Stock_index[Stock_index.Index == i].reset_index(drop=True)
        handler.增加技術指標MA(s,day,labels[l],test_)
        
    Stock_index[features[l]] = test_

features =['MA5 slop', 'MA20 slop' ]
labels = ['MA5', 'MA20'] 
for l in range(len(labels)):
    handler.Slope (Stock_index , features[l] , labels[l] )

features =['乖離5', '乖離20' ]
label1 = ['收盤','收盤']
label2 = ['MA5','MA20']
for i in range(len(labels)):
    handler.distance (Stock_index , features[i] , label1[i] ,label2[i] )
    
Stock_index

,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
0,未含電子指數,15054.77,65.44,0.44,20190429,0.000,0.0000,0.0000,0.0000,1.00,1.00
1,未含電子指數,15107.62,52.85,0.35,20190430,0.000,0.0000,0.0000,0.0000,1.00,1.00
2,未含電子指數,15197.27,89.65,0.59,20190502,0.000,0.0000,0.0000,0.0000,1.00,1.00
3,未含電子指數,15194.64,2.63,-0.02,20190503,0.000,0.0000,0.0000,0.0000,1.00,1.00
4,未含電子指數,15004.13,190.51,-1.25,20190506,0.000,0.0000,0.0000,0.0000,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...
1523,電子類兩倍槓桿指數,42609.98,546.62,-1.27,20210409,50370.074,42376.9040,0.5522,0.5831,-0.18,0.01
1524,電子類兩倍槓桿指數,41895.54,714.44,-1.68,20210412,50721.934,42573.8155,0.6985,0.4647,-0.21,-0.02
1525,電子類兩倍槓桿指數,41628.77,266.77,-0.64,20210413,50810.128,42678.0805,0.1739,0.2449,-0.22,-0.03
1526,電子類兩倍槓桿指數,41811.03,182.26,0.44,20210414,50730.660,42762.7490,-0.1564,0.1984,-0.21,-0.02


In [86]:
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
Market

,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
479,發行量加權股價指數,10145.28,80.50,0.80,20190218,18776.012,16288.8305,-1.0701,0.1385,-0.85,-0.61
480,發行量加權股價指數,10152.26,6.98,0.07,20190219,18654.224,16322.7085,-0.6486,0.2080,-0.84,-0.61
481,發行量加權股價指數,10272.46,120.20,1.18,20190220,18616.926,16365.6095,-0.1999,0.2628,-0.81,-0.59
482,發行量加權股價指數,10319.53,47.07,0.46,20190221,18639.420,16404.1740,0.1208,0.2356,-0.81,-0.59
483,發行量加權股價指數,10322.92,3.39,0.03,20190222,18640.374,16440.3285,0.0051,0.2204,-0.81,-0.59
...,...,...,...,...,...,...,...,...,...,...,...
998,發行量加權股價指數,16854.10,72.34,-0.43,20210409,20584.354,17567.8190,0.3338,0.2836,-0.22,-0.04
999,發行量加權股價指數,16859.70,5.60,0.03,20210412,20699.326,17628.6055,0.5585,0.3460,-0.23,-0.05
1000,發行量加權股價指數,16824.91,34.79,-0.21,20210413,20814.030,17681.8225,0.5541,0.3019,-0.24,-0.05
1001,發行量加權股價指數,16865.97,41.06,0.24,20210414,20918.556,17737.7735,0.5022,0.3164,-0.24,-0.05


In [ ]:
Stock.columns

In [ ]:
Stock

In [ ]:
list_ma5 = []
list_ma20 = []
list_market = []
list_5D = []
list_20D = []
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
for i in range(len(Stock)):
    day = Stock.date[i].squeeze()
    market = Market[(Market.date == int(day))]
    list_ma5.append(market['MA5 slop'].squeeze())
    list_ma20.append(market['MA20 slop'].squeeze())
    list_market.append(market.漲跌.squeeze())
    list_5D.append(market.乖離5.squeeze())
    list_20D.append(market.乖離20.squeeze())
    

Stock['大盤漲跌'] = list_market
Stock['大盤5MA'] = list_ma5
Stock['大盤20MA'] = list_ma20
Stock['大盤5乖離'] = list_5D
Stock['大盤20乖離'] = list_20D

In [82]:
Stock.to_csv('model.csv', index=False)